# Setup

In [ ]:
from __future__ import annotations
from model import interval, HourlySeries, Dummy
from estimation import IV_results, IVOptions, ModelResults, MultipleModelResults, ModelInputParams, ar_model_fit
from estimation import exog_list
from synthetic import Country
from database import Areas, PriceArea

from matplotlib.axes import Axes
from matplotlib.figure import Figure
from matplotlib.table import Table
import mpl_toolkits.axisartist as axisartist

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from functools import lru_cache
from typing import Literal

import statsmodels.tsa.ar_model as ar_model

from plot_tools import *
from statsmodels.api import OLS


pd.options.plotting.backend = "plotly"



In [ ]:
AREA = Areas.DE2

INTERVAL = interval(
    start=pd.Timestamp("201711010000"),    # First nat.gas point
    end=pd.Timestamp("202012312300"),      # End
)

DEFAULT_SERIES = HourlySeries(INTERVAL)
DEFAULT_DUMMY = Dummy(INTERVAL)

In [ ]:
# Define the start and end timestamps
start_timestamp = pd.Timestamp("2017-11-01 00:00")
end_timestamp = pd.Timestamp("2020-12-31 23:00")

# Calculate the midpoint
midpoint = start_timestamp + (end_timestamp - start_timestamp) / 2
print(midpoint)

In [ ]:
COUNTRY = Country(AREA, DEFAULT_SERIES, remove_holidays=True)

# Main Figure

Main Figure

In [ ]:
models=[
        ModelInputParams(IVOptions.REGULAR, order=0),
        
        ModelInputParams(IVOptions.CONDITIONAL_WIND, order=50),
        ModelInputParams(IVOptions.CONDITIONAL_DEMAND, order=1, order_w=0),
        ModelInputParams(IVOptions.CONDITIONAL_H0, order=50),

        ModelInputParams(IVOptions.TRUNCATED_NUISANCE_ORDER, order=50, order_d=1),
        ModelInputParams(IVOptions.CLEAN_2dim_ORDER, order=50, order_price=1),
        ModelInputParams(IVOptions.TRUNCATED_IV_2dim_ORDER, order=50, order_price=1),
        ModelInputParams(IVOptions.IV_2dim_ORDER, order=50, order_price=1),
        ]

def plot_transposed_panel(
    ax: axisartist.Axes,
    price_area:PriceArea,
    models: list[ModelInputParams] = models,
    controls: bool = False,
    title:str="",
    interval=INTERVAL  , 
    log_log=False,
    minimal_controls=False,
    split_gas:Literal[False, "high", "low"]=False,
    subset_months:list[int] = [],
    start_hour=None,
    end_hour=None,
    drop_holidays:bool=False
):
    
    demand = COUNTRY.consumption[COUNTRY.consumption.index.isin(interval)]
    endog = COUNTRY.price[COUNTRY.price.index.isin(interval)]
    instrument = COUNTRY.wind_generation[COUNTRY.wind_generation.index.isin(interval)]
    series = HourlySeries(interval)
    dummy = Dummy(interval)

    axs0 = ax


    axs0.set_clip_on(False)
    
    Y = len(models)
    y = len(models)
    first_p0 = True
    first_p1 = True
    civ_wind_results = None
    method_labels = {}

    for mod in models:
        print(f"Starting with {mod.strategy}")

        if isinstance(mod.strategy, IVOptions):
            result: ModelResults = IV_results(
                price_area=price_area,
                demand=demand,
                price=endog,
                wind=instrument,
                dummy=dummy,
                series=series,
                model=mod,
                n_lags=26,
                controls=controls,
                log_log=log_log,
                minimal_controls=minimal_controls,
                split_gas=split_gas,
                subset_months=subset_months,
                start_hour=start_hour,
                end_hour=end_hour,
                drop_holidays=drop_holidays
            )

        else:
            raise Exception("One of the models is not specified correctly")
        
        if isinstance(result, MultipleModelResults):
            for k in result.estimate.keys():
                if k == '_P0':
                    ci = result.ci['_P0']
                    axs0.errorbar(
                        result.estimate['_P0'],
                        y,
                        xerr=ci[1],
                        fmt="o",
                        capsize=5,
                        color="darkorange",
                        label="Own-price elasticity ($P_t\\rightarrow D_t$)" if first_p0 else ""
                    )
                    method_labels[y] = result.latex_name
                    first_p0 = False
                elif k == '_P1':
                    ci = result.ci['_P0']
                    axs0.errorbar(
                        result.estimate['_P1'],
                        y,
                        xerr=ci[1],
                        fmt="^",
                        capsize=5,
                        color="dodgerblue",
                        label="Cross-price elasticity ($P_{t-1}\\rightarrow D_t$)" if first_p1 else ""
                    )
                    first_p1 = False
        else:
            axs0.errorbar(
                result.estimate,
                y,
                xerr=result.ci[1],
                fmt="o",
                capsize=5,
                color="darkorange"
                
            )
            method_labels[y] = result.latex_name

        y -= 1

        if mod.strategy == IVOptions.CONDITIONAL_WIND:
            civ_wind_results = result

        print(f"Estimate = {result.estimate}")

    assert isinstance(civ_wind_results, ModelResults) and not isinstance(civ_wind_results, MultipleModelResults), "A CIV-Wind strategy is required!"

    axs0.set_yticks(list(method_labels.keys()))
    axs0.set_yticklabels([f"#{len(method_labels)-k+1}   "+v for k, v in method_labels.items()])
    axs0.axis["left"].major_ticklabels.set_ha("left")
    axs0.axis["left"].major_ticks.set_ticksize(0)
    axs0.axis["right"].major_ticks.set_ticksize(0)
    axs0.yaxis.grid(visible=False)
    civ_wind_estimate = civ_wind_results.estimate
    axs0.set_xlim((2.5*civ_wind_estimate, -2.5*civ_wind_estimate))
    axs0.set_ylim(0.5, Y + 0.5)
    axs0.set_title(title)


    axs0.fill_betweenx(range(0, Y+2), civ_wind_results.estimate+civ_wind_results.ci[0], civ_wind_results.estimate+civ_wind_results.ci[1],
                       color='darkorange', alpha=0.2)
    axs0.axvline(x=civ_wind_results.estimate, ls='--', color='darkorange', lw=2, alpha=0.9)
    

    axs0.axvline(x=0, ls='-', color='k', lw=1)
    for h in range(0, Y+1):
        axs0.axhline(y=h+0.5, ls='-', color='grey', xmin=-0.8, xmax=1, clip_on=False)

    axs0.set_xlabel("estimated slope of the demand curve" if not log_log else "estimated elasticity", size="large")



In [ ]:
def main_figure(models:list[ModelInputParams], price_area:PriceArea,  name:str, controls:bool, log_log:bool, start_hour:int|None, end_hour:int|None, drop_holidays:bool):
    fig, axs = plt.subplots(1, 1, sharex=False,  sharey=True, figsize=(WIDE_WIDTH, HEIGHT), subplot_kw={'axes_class': axisartist.Axes})

    plot_transposed_panel(axs, models=models, price_area=price_area, controls=controls, title="Estimates", log_log=log_log, start_hour=start_hour, end_hour=end_hour, drop_holidays=drop_holidays)

    plot_table(fig)

    fig.tight_layout()

    fig.text(0.16, 0.91, "Estimators", fontsize="large", va='top')

    if log_log:
        fig.legend(loc="upper center", frameon=False, bbox_to_anchor=(0.5, 0), fancybox=True, fontsize='large')

    save_figure(fig, name, VERSION)

In [ ]:
main_figure(models=models, price_area=AREA, name="main", controls=True, log_log=False, start_hour=None, end_hour=None, drop_holidays=False)

Log-log

In [ ]:
main_figure(models=models, price_area=AREA, name="main_log_log", controls=True, log_log=True, start_hour=None, end_hour=None, drop_holidays=False)


Subset hours

In [ ]:
main_figure(models=models, price_area=AREA, name="8_to_20", controls=True, log_log=False, start_hour=8, end_hour=20, drop_holidays=False)
main_figure(models=models, price_area=AREA, name="8_to_20_log_log", controls=True, log_log=True, start_hour=8, end_hour=20, drop_holidays=False)


In [ ]:
main_figure(models=models, price_area=AREA, name="20_to_8", controls=True, log_log=False, start_hour=20, end_hour=8, drop_holidays=False)
main_figure(models=models, price_area=AREA, name="20_to_8_log_log", controls=True, log_log=True, start_hour=20, end_hour=8, drop_holidays=False)

Gas split

In [ ]:
fig, axs = plt.subplots(1, 1, sharex=False,  sharey=True, figsize=(WIDE_WIDTH, HEIGHT))

plot_transposed_panel(axs, models=models, price_area=AREA, controls=True, split_gas='high')

fig.tight_layout()

save_figure(fig, f"high_gas", VERSION)

In [ ]:
fig, axs = plt.subplots(1, 1, sharex=False,  sharey=True, figsize=(WIDE_WIDTH, HEIGHT))

plot_transposed_panel(axs, models=models, price_area=AREA, controls=True, split_gas='low')

fig.tight_layout()

save_figure(fig, f"low_gas", VERSION)

Minimal controls

In [ ]:
fig, axs = plt.subplots(1, 1, sharex=True,  sharey=True, figsize=(WIDE_WIDTH, HEIGHT))

plot_transposed_panel(axs, models=models, price_area=AREA, controls=True, minimal_controls=True)

plot_table(fig)
fig.tight_layout()

save_figure(fig, f"minimal_controls", VERSION)

In [ ]:
fig, axs = plt.subplots(1, 1, sharex=True,  sharey=True, figsize=(WIDE_WIDTH, HEIGHT))

plot_transposed_panel(axs, models=models, price_area=AREA, controls=True, minimal_controls=True, log_log=True)

plot_table(fig)
fig.tight_layout()
fig.legend(loc="upper center", frameon=False, bbox_to_anchor=(0.5, 0), fancybox=True, fontsize='large')

save_figure(fig, f"minimal_controls_log_log", VERSION)

Subset months

In [ ]:
fig, axs = plt.subplots(1, 1, sharex=False,  sharey=True, figsize=(WIDE_WIDTH, HEIGHT))

plot_transposed_panel(axs, models=models, price_area=AREA, controls=True, subset_months=[12, 1, 2])

fig.tight_layout()

save_figure(fig, f"sensitivity_winter", VERSION)

In [ ]:
fig, axs = plt.subplots(1, 1, sharex=False,  sharey=True, figsize=(WIDE_WIDTH, HEIGHT))

plot_transposed_panel(axs, models=models, price_area=AREA, controls=True, subset_months=[3, 4, 5])

fig.tight_layout()

save_figure(fig, f"sensitivity_spring", VERSION)

In [ ]:
fig, axs = plt.subplots(1, 1, sharex=False,  sharey=True, figsize=(WIDE_WIDTH, HEIGHT))

plot_transposed_panel(axs, models=models, price_area=AREA, controls=True, subset_months=[6, 7, 8])

fig.tight_layout()

save_figure(fig, f"sensitivity_summer", VERSION)

In [ ]:
fig, axs = plt.subplots(1, 1, sharex=False,  sharey=True, figsize=(WIDE_WIDTH, HEIGHT))

plot_transposed_panel(axs, models=models, price_area=AREA, controls=True, subset_months=[9, 10, 11])

fig.tight_layout()

save_figure(fig, f"sensitivity_fall", VERSION)

# Lag selection

Everything beyond this point is no longer used in the current state of the paper and its completeness is not ensured

Relevant lags inspection

In [ ]:
def get_most_relevant_lags(ar_params:pd.Series, n:int) -> list[int]:
    params_without_const = ar_params.drop('const')
    abs_params = params_without_const.abs()
    sorted_params = abs_params.sort_values(ascending=False)

    top_n_lags = sorted_params.head(n)
    print(top_n_lags)
    
    try:
        top_n_lags = [int(l.split(".L")[1]) for l in top_n_lags.index]
        return top_n_lags
    except IndexError as e:
        print(top_n_lags)
        raise e



In [ ]:
from model import ModelData, IVModel

def get_custom_CIV_wind_estimate(
    demand: pd.Series,
    price: pd.Series,
    wind: pd.Series,
    selected_lags: list[int],
    series: HourlySeries,
    dummy: Dummy,
    price_area: PriceArea,
    cov_type: str = "kernel",
) -> ModelResults:

    instrument = [wind]
    endog = [price]

    exog = exog_list(
        series=series, dummy=dummy, price_area=price_area, minimal=False
    ).copy()
    
    
    for i in selected_lags:
        shift_instrument = wind.copy().shift(i)
        shift_instrument.name = f"instrument shifted {i}"
        exog.append(shift_instrument)

       
    data = ModelData.construct(
        dependent=demand,
        exogenous=exog,
        endogenous=endog,
        instruments=instrument,
        scaling=False,
    )

    iv_model = IVModel(data=data, cov_type=cov_type)
    fitted: IVResults = iv_model.fitted_model  # type: ignore

    mse = 0

    price_coeff = fitted.params["price"]
    price_ci: tuple[float, float] = tuple(fitted.conf_int(level=0.95).loc["price"] - price_coeff)  # type: ignore
    return ModelResults(
        estimate=price_coeff,
        ci=price_ci,
        model="Custom_CIV_Wind",
        latex_name="Custom CIV Wind",
        f_statistic=fitted.first_stage.diagnostics["f.stat"]["price"],
        mse=mse,
    )

In [ ]:

def get_custom_residual_IV_estimate(
    demand: pd.Series,
    price: pd.Series,
    wind: pd.Series,
    selected_lags: list[int],
    series: HourlySeries,
    dummy: Dummy,
    price_area: PriceArea,
    cov_type: str = "kernel",
) -> ModelResults:

    instrument = [wind]
    endog = [price]

    exog_df = pd.concat(
                (exog_list(series=series, dummy=dummy, price_area=price_area, minimal=False, wind=True).copy() + 
                 [wind.copy().shift(i).rename(f"wind_t-{i}") for i in selected_lags]), axis=1
                )
    exog_df = exog_df.loc[exog_df.index.isin(wind.index)]
    residuals = sm.OLS(wind, exog=exog_df, missing="drop").fit().resid.rename("wind_generation_resid")
    
    instrument = [residuals]

    exog = exog_list(
        series=series, dummy=dummy, price_area=price_area, minimal=False
    ).copy()
       
    data = ModelData.construct(
        dependent=demand,
        exogenous=exog,
        endogenous=endog,
        instruments=instrument,
        scaling=False,
    )

    iv_model = IVModel(data=data, cov_type=cov_type)
    fitted: IVResults = iv_model.fitted_model  # type: ignore

    mse = 0

    price_coeff = fitted.params["price"]
    price_ci: tuple[float, float] = tuple(fitted.conf_int(level=0.95).loc["price"] - price_coeff)  # type: ignore
    return ModelResults(
        estimate=price_coeff,
        ci=price_ci,
        model="Custom_Residual_IV",
        latex_name="Custom Residual IV",
        f_statistic=fitted.first_stage.diagnostics["f.stat"]["price"],
        mse=mse,
    )

In [ ]:
def solar(price_area, series:HourlySeries):
    if (price_area == Areas.DE1 or price_area == Areas.DE2) and (
        series.timestamps.min() < pd.Timestamp("201809302300", tz="Europe/Berlin")
        and pd.Timestamp("201810010000", tz="Europe/Berlin")
        < series.timestamps.max()
    ):
        series1 = HourlySeries(
            interval(
                start=series.timestamps.min(),
                end=pd.Timestamp("201809302300", tz="Europe/Berlin"),
            )
        )
        series2 = HourlySeries(
            interval(
                start=pd.Timestamp("201810010000", tz="Europe/Berlin"),
                end=series.timestamps.max(),
            )
        )

        solar_generation = pd.concat(
            [
                series1.solar_generation(price_area=Areas.DE1).div(1000),
                series2.solar_generation(price_area=Areas.DE2).div(1000),
            ]
        ).rename("generation.DE.Solar")
    else:
        solar_generation = series.solar_generation(price_area)

    return solar_generation

In [ ]:
def run_succesive_civ_wind(lags_to_choose_from:int, number_of_estimates:int):
    assert number_of_estimates <= lags_to_choose_from
    print("Launch")

    series = DEFAULT_SERIES
    dummy = DEFAULT_DUMMY
    price_area = AREA
    exog_df = pd.concat(exog_list(series=series, dummy=dummy, price_area=price_area, minimal=False), axis=1)
    exog_df = pd.concat(dummy.month()+dummy.hour()+dummy.year()+[dummy.constant(),
        series.hdd(price_area), series.cdd(price_area), series.sunlight(price_area), solar(price_area, series)
        ], axis=1)
    
    exog_df = exog_df.loc[exog_df.index.isin(INTERVAL)]

    demand = COUNTRY.consumption.loc[COUNTRY.consumption.index.isin(INTERVAL)]
    price = COUNTRY.price.loc[COUNTRY.price.index.isin(INTERVAL)]
    wind = COUNTRY.wind_generation.loc[COUNTRY.wind_generation.index.isin(INTERVAL)]
    
    # print(exog_df)

    ols_resid = OLS(endog=wind, exog=exog_df).fit().resid
    print("OLS fit")
    params = ar_model_fit(series=ols_resid, lags=lags_to_choose_from, exog=None).params


    Y = number_of_estimates+1
    y = number_of_estimates+1
    civ_wind_results = None
    method_labels = {}

    fig, ax = plt.subplots(1, 1, figsize=(WIDE_WIDTH, HEIGHT*2), subplot_kw={'axes_class': axisartist.Axes})

    civ_wind_results = IV_results(demand=demand, 
                        price=price,
                        wind=wind,
                        series=series,
                        dummy=dummy,
                        model=ModelInputParams(IVOptions.CONDITIONAL_WIND, 50),
                        controls=True,
                        price_area=price_area)

    ax.errorbar(
        civ_wind_results.estimate,
        y,
        xerr=civ_wind_results.ci[1],
        fmt="o",
        capsize=5,
        color="darkorange"
        
    )

    method_labels[y] = civ_wind_results.latex_name

    y -= 1

    print(f"{civ_wind_results.latex_name} = {civ_wind_results.estimate}")
    
    for i in range(1, number_of_estimates+1):
        print(f"Starting with {i} lags")
        selected_lags = get_most_relevant_lags(ar_params=params, n=i)
        result = get_custom_CIV_wind_estimate(demand=demand, 
                                            price=price,
                                            wind=wind,
                                            selected_lags=selected_lags,
                                            series=series,
                                            dummy=dummy,
                                            price_area=price_area)

        ax.errorbar(
            result.estimate,
            y,
            xerr=result.ci[1],
            fmt="o",
            capsize=5,
            color="darkorange"
            
        )
        print(f"CIV-Wind estimate adding lag {selected_lags[-1]} = {result.estimate}")

        result = get_custom_residual_IV_estimate(demand=demand, 
                                            price=price,
                                            wind=wind,
                                            selected_lags=selected_lags,
                                            series=series,
                                            dummy=dummy,
                                            price_area=price_area)

        ax.errorbar(
            result.estimate,
            y,
            xerr=result.ci[1],
            fmt="o",
            capsize=5,
            color="dodgerblue"
            
        )
        method_labels[y] = selected_lags[-1]

        y -= 1


        print(f"Residual IV estimate adding lag {selected_lags[-1]} = {result.estimate}")


    ax.set_yticks(list(method_labels.keys()))
    ax.set_yticklabels([f"({i+1}) Added lag {v}" if i != 0 else f"({i+1}) {v}" for i, (k, v) in enumerate(method_labels.items())])
    ax.axis["left"].major_ticklabels.set_ha("right")
    ax.axis["left"].major_ticks.set_ticksize(0)
    ax.axis["right"].major_ticks.set_ticksize(0)
    ax.yaxis.grid(visible=False)
    
    ax.set_ylim(0.5, Y + 0.5)
    ax.set_title("")

    civ_wind_estimate = civ_wind_results.estimate
    ax.set_xlim((2.5*civ_wind_estimate, -2.5*civ_wind_estimate))
    ax.fill_betweenx(range(0, Y+2), civ_wind_results.estimate+civ_wind_results.ci[0], civ_wind_results.estimate+civ_wind_results.ci[1],
                       color='darkorange', alpha=0.2)
    ax.axvline(x=civ_wind_results.estimate, ls='--', color='darkorange', lw=2, alpha=0.9)
    

    ax.axvline(x=0, ls='-', color='k', lw=1)
    for h in range(0, Y+1):
        ax.axhline(y=h+0.5, ls='-', color='grey', xmin=-0.15, xmax=1, clip_on=False)

    ax.set_xlabel("Estimated slope of the demand curve", size="large")
    plt.show()




In [ ]:
run_succesive_civ_wind(50, 20)

In [ ]:
# run_succesive_civ_wind(lags_to_choose_from=170, 
#                        number_of_estimates=25)


PACF

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import pacf

def pacf_plot(nlags:int):
    series = DEFAULT_SERIES
    dummy = DEFAULT_DUMMY
    price_area = AREA
    exog_df = pd.concat(exog_list(series=series, dummy=dummy, price_area=price_area, minimal=False), axis=1)
    exog_df = pd.concat(dummy.month()+dummy.hour()+dummy.year()+[dummy.constant(),
        series.hdd(price_area), series.cdd(price_area), series.sunlight(price_area), solar(price_area, series)
        ], axis=1)

    exog_df = exog_df.loc[exog_df.index.isin(INTERVAL)]

    demand = COUNTRY.consumption.loc[COUNTRY.consumption.index.isin(INTERVAL)]
    price = COUNTRY.price.loc[COUNTRY.price.index.isin(INTERVAL)]
    wind = COUNTRY.wind_generation.loc[COUNTRY.wind_generation.index.isin(INTERVAL)]

    # print(exog_df)

    ols_resid = OLS(endog=wind, exog=exog_df).fit().resid
    print("OLS fit")

    # Plot PACF with confidence intervals
    plt.figure(figsize=(8, 3))
    # acf_x, confint = pacf(ols_resid, nlags=nlags, method='ywm', alpha=0.05)
    # print([ci[1]-ci[0] for i, ci in enumerate(confint)])
    # confint = [ci[1]-acf_x[i] for i, ci in enumerate(confint)]
    # plt.errorbar(list(range(nlags+1)), acf_x, confint, color="dodgerblue", capsize=5)
    plot_pacf(ols_resid, lags=170, method='ywm', alpha=0.05)  # You can adjust the number of lags
    plt.xlabel('Lags')
    plt.ylabel('Partial Autocorrelation')
    plt.title('Partial Autocorrelation Function (PACF)')
    plt.show()


In [ ]:
pacf_plot(12)

In [ ]:
pacf_plot(26)

In [ ]:
pacf_plot(50)

In [ ]:
pacf_plot(170)

AIC

In [ ]:
@lru_cache
def wind_resid():
    series = DEFAULT_SERIES
    dummy = DEFAULT_DUMMY
    price_area = AREA
    exog_df = pd.concat(exog_list(series=series, dummy=dummy, price_area=price_area, minimal=False), axis=1)
    exog_df = pd.concat(dummy.month()+dummy.hour()+dummy.year()+[dummy.constant(),
        series.hdd(price_area), series.cdd(price_area), series.sunlight(price_area), solar(price_area, series)
        ], axis=1)

    exog_df = exog_df.loc[exog_df.index.isin(INTERVAL)]

    demand = COUNTRY.consumption.loc[COUNTRY.consumption.index.isin(INTERVAL)]
    price = COUNTRY.price.loc[COUNTRY.price.index.isin(INTERVAL)]
    wind = COUNTRY.wind_generation.loc[COUNTRY.wind_generation.index.isin(INTERVAL)]

    # print(exog_df)

    ols_resid = OLS(endog=wind, exog=exog_df).fit().resid
    print("OLS fit")
    return ols_resid

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.ar_model import AutoReg
from simulations import Simulation


def AIC_for_p_lags(p:int, report:bool=False):
    # Example ARIMA(p, d, q) where p is the lag order determined from PACF
    exog = pd.concat([wind_resid().shift(i).rename(f"squared_wind_t-{i}")**2 for i in range(1, p+1)], axis=1)
    wind_res = OLS(wind_resid(), exog, missing="drop", hasconst=False).fit().resid
    model = AutoReg(wind_res, lags=p, trend='n', missing="drop")  # Specify your (p,d,q) orders
    fitted_model = model.fit()
    aic = round(fitted_model.aic, 2)
    if report:
        print(f"AR{p} fit")
        print(f'AR{p} Model AIC: {aic}')

    return aic


def plot_improvement_aic(lag_list:list[int]):
    aic_list = []
    # aic_list.append(AIC_for_p_lags(0))
    for lag in lag_list:
        aic_list.append(AIC_for_p_lags(lag))
        # print(f"AIC improvement = {round(aic_list[-2]-aic_list[-1], 2)}")
        
    # aic_list.pop(0)
    plt.figure(figsize=(8, 3))
    plt.plot(lag_list, aic_list)



In [ ]:
plot_improvement_aic([2+24*i for i in range(0, 61)])


In [ ]:
plot_improvement_aic([i for i in range(1, 170)])


In [ ]:
demand = COUNTRY.consumption.loc[COUNTRY.consumption.index.isin(INTERVAL)]
price = COUNTRY.price.loc[COUNTRY.price.index.isin(INTERVAL)]
wind = COUNTRY.wind_generation.loc[COUNTRY.wind_generation.index.isin(INTERVAL)]
ex_l = [wind]+exog_list(series=DEFAULT_SERIES, dummy=DEFAULT_DUMMY, price_area=AREA, minimal=False)
for i in range(1, 51):
    ex_l.append(wind.shift(i).rename(f"Wind_t-{i}"))
exog_df = pd.concat(ex_l, axis=1)


exog_df = exog_df.loc[exog_df.index.isin(INTERVAL)]
exog_df = exog_df.dropna()
new_index:pd.DatetimeIndex = exog_df.index

demand = COUNTRY.consumption.loc[COUNTRY.consumption.index.isin(new_index)]
price = COUNTRY.price.loc[COUNTRY.price.index.isin(new_index)]
wind = COUNTRY.wind_generation.loc[COUNTRY.wind_generation.index.isin(new_index)]

# print(exog_df)

ols_resid = OLS(endog=demand, exog=exog_df).fit()

In [ ]:
ols_resid.summary()